<a href="https://colab.research.google.com/github/SurajitBiswasPhysics/surajit-biswas.github.io/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-13 16:07:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  85.5MB/s    in 0.3s    

2024-12-13 16:07:10 (85.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [12]:
# Count the number of ratings per user and per book
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Filter users with less than 200 ratings and books with less than 100 ratings
filtered_ratings = df_ratings[
    df_ratings['user'].isin(user_counts[user_counts >= 200].index) &
    df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)
]

# Merge book data with ratings to get book titles
df_ratings_merged = pd.merge(filtered_ratings, df_books, on='isbn', how='left')

# Create a pivot table: rows = users, columns = books, values = ratings
book_user_matrix = df_ratings_merged.pivot_table(index='user', columns='title', values='rating', fill_value=0)

# Convert the pivot table to a sparse matrix (for efficiency)

sparse_matrix = csr_matrix(book_user_matrix.values)


In [13]:
# Fit the K-Nearest Neighbors model
knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='cosine')
knn_model.fit(sparse_matrix)

NearestNeighbors(metric='cosine')

In [45]:
def get_recommends(book_title=""):
    # Check if the book title exists in the dataset
    if book_title not in book_user_matrix.columns:
        return f"Book '{book_title}' not found in the dataset."

    # Find the index of the book in the user-item matrix
    book_idx = book_user_matrix.columns.get_loc(book_title)

    # Extract the ratings vector for the book (1D array)
    book_ratings = sparse_matrix[:, book_idx].T  # Shape will be (1, n_users)

    # Ensure compatibility with the trained model
    book_ratings = book_ratings.toarray().reshape(1, -1)  # Convert to dense and reshape to (1, n_features)

    # Get the nearest neighbors, including the book itself
    distances, indices = knn_model.kneighbors(book_ratings, n_neighbors=6)

    # Collect recommendations (excluding the book itself)
    recommended_books = []
    for dist, idx in zip(distances.flatten()[1:], indices.flatten()[1:]):
        recommended_books.append([book_user_matrix.columns[idx], dist])

    # Return the final output format
    return [book_title, recommended_books]


Shape of sparse_matrix: (888, 673)


NameError: name 'book_ratings' is not defined

In [46]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

ValueError: X has 888 features, but NearestNeighbors is expecting 673 features as input.

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()